The Goal here is to take a folder of images (you can change the path to this folder in the code below) and process them into a csv file that has 48 values for every image (image is 256X256 and it is divided into 16 blocks of 64X64 , for every block 3 values-mode value for Red,mode value for Green, mode value for Blue in RGB, so a total of 16blocks X 3values = 48values + 1output_label (0 for class 1 and 1 for class 2 as an output label))

Importing all the dependencies

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.neural_network as skn
from sklearn.model_selection import train_test_split
import os
from google.colab.patches import cv2_imshow
import cv2
import csv
from collections import Counter

Taking all the images from the folder into images[]

**CSV FILE CREATION**

I have used a dataset having 100 images of agriculture (0 output label) and 100 images of airplane (1 otuput label). This is a subset of the dataset taken from https://www.kaggle.com/datasets/abdulhasibuddin/uc-merced-land-use-dataset
The dataset folder has first 100 images of agriculture and next 100 of airplane.

**Note**:You can use your own dataset, just change the path to the dataset folder in the code below

In [ ]:
directory = '/content/drive/MyDrive/Colab Notebooks/Data/Dataset'

images=[]
for filename in os.listdir(directory):
    images.append(cv2.imread('/content/drive/MyDrive/Colab Notebooks/Data/Dataset/'+filename))

Basically i created a list of images

Checking the shapes

In [ ]:
# print(images.size())
type(images[0])

numpy.ndarray

Now traversing through all the blocks and storing the data in a dictionary and then converted to a dataframe and finally to a csv

In [ ]:
with open('/content/drive/MyDrive/Colab Notebooks/Data/airplanes_agriculture_dataset', 'w', newline='') as file:
  writer=csv.writer(file)
  count=0
  for image in images:

    #Creating a list of parameters to write for each image
    data=[]

    for row in range(0,4):
      for col in range(0,4):

        #Taking one block at a time
        block=image[row*64:row*64+64,col*64:col*64+64]

        counter_red=Counter()
        counter_green=Counter()
        counter_blue=Counter()

        for block_row in block:
          for pixel in block_row:
            counter_red[pixel[0]]+=1
            counter_green[pixel[1]]+=1
            counter_blue[pixel[2]]+=1

        #Now the counter stores the map of r's g's and b's in a block
        mode_red=counter_red.most_common()[0][0]
        mode_green=counter_green.most_common()[0][0]
        mode_blue=counter_blue.most_common()[0][0]

        data.append(mode_red)
        data.append(mode_green)
        data.append(mode_blue)
        #Now we will move on to the next block and hence the data will be filled up with 16*3 values
    if (count<=99):
      data.append(0)
    else:
      data.append(1)
    count+=1

    #Now for the final act, we will write a row in the csv
    print(data)
    writer.writerow(data)


[115, 120, 116, 122, 123, 122, 116, 121, 116, 112, 116, 114, 114, 123, 116, 115, 119, 115, 117, 125, 116, 117, 116, 116, 114, 122, 116, 111, 120, 118, 114, 125, 113, 110, 120, 112, 117, 125, 121, 120, 129, 116, 115, 123, 126, 110, 123, 120, 0]
[115, 121, 116, 103, 123, 118, 111, 119, 119, 109, 116, 115, 114, 116, 119, 110, 117, 125, 107, 120, 116, 105, 122, 116, 109, 120, 120, 109, 124, 120, 107, 121, 120, 115, 121, 119, 112, 124, 113, 114, 119, 111, 105, 127, 117, 114, 123, 117, 0]
[103, 120, 128, 111, 120, 115, 109, 118, 107, 108, 117, 107, 121, 126, 124, 111, 119, 122, 113, 121, 115, 113, 121, 110, 116, 121, 114, 109, 121, 112, 112, 119, 116, 115, 119, 115, 122, 127, 127, 119, 122, 116, 111, 114, 112, 118, 125, 125, 0]
[193, 210, 220, 191, 209, 225, 194, 214, 236, 185, 43, 218, 140, 156, 253, 122, 135, 253, 149, 168, 184, 102, 125, 142, 123, 132, 138, 150, 168, 179, 148, 165, 178, 24, 40, 60, 160, 176, 191, 156, 173, 185, 149, 163, 176, 120, 139, 158, 0]
[192, 225, 216, 192, 225, 21

Now the csv file is ready to be used for MLP Classification

Using **MLP Classifier**

Importing the dependencies

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
import pandas as pd
import numpy as np

In [ ]:
csv_file=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/airplanes_agriculture_dataset')
x_data=csv_file.iloc[:,0:-1].values
y_data=csv_file.iloc[:,-1].values
print("Shape of X-Data:",x_data.shape)
print("Shape of Y-Data:",y_data.shape)

Shape of X-Data: (199, 48)
Shape of Y-Data: (199,)


Now making the train test split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.3,shuffle=True)
print("x_train.shape:",x_train.shape)
print("y_train.shape",y_train.shape)
print("x_test.shape:",x_test.shape)
print("y_test.shape:",y_test.shape)

x_train.shape: (139, 48)
y_train.shape (139,)
x_test.shape: (60, 48)
y_test.shape: (60,)


Now training the MLP Classifier

The default values for the MLPclassifier are ,

class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)

In [ ]:
classifier=MLPClassifier(hidden_layer_sizes=(100,50,10),learning_rate_init=0.0001,max_iter=600)
classifier.fit(x_train,y_train)
prediction=classifier.predict(x_test)
print("Prediction.shape:",prediction.shape)

Prediction.shape: (60,)


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (600) reached and the optimization hasn't converged yet.
  warnings.warn(


Now checking the accuracy

In [ ]:
correct=0
for i in range(y_test.shape[0]):
  if (prediction[i]==y_test[0]):
    correct+=1

print(correct)

26


In [ ]:
print("Accuracy:",correct*100/prediction.shape[0])

Accuracy: 43.333333333333336


Now trying to implement **FFNN** with pytorch

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

Creating the model for **FFNN**

In [ ]:
# Load the Excel file with the features and class labels
data_path = "/content/drive/MyDrive/Colab Notebooks/Data/airplanes_agriculture_dataset"
df = pd.read_csv(data_path)

X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


y_train = to_categorical(y_train, num_classes=2)
y_test = to_categorical(y_test, num_classes=2)


model = Sequential()

model.add(Dense(32, activation='relu', input_shape=(48,)))

model.add(Dense(16, activation='relu'))

model.add(Dense(2, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=100, batch_size=10, validation_data=(X_test, y_test))

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")


y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)


from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.4803 - loss: 1.0806 - val_accuracy: 0.5500 - val_loss: 0.7574
Epoch 2/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4883 - loss: 0.8322 - val_accuracy: 0.6000 - val_loss: 0.6801
Epoch 3/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5671 - loss: 0.7029 - val_accuracy: 0.6250 - val_loss: 0.6515
Epoch 4/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5767 - loss: 0.6871 - val_accuracy: 0.6000 - val_loss: 0.6424
Epoch 5/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6524 - loss: 0.6228 - val_accuracy: 0.5750 - val_loss: 0.6398
Epoch 6/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6726 - loss: 0.5893 - val_accuracy: 0.6250 - val_loss: 0.6370
Epoch 7/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6396 - loss: 0.5327 - val_accuracy: 0.6250 - val_loss: 0.6253
Epoch 8/100
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7783 - loss: 0.4695 - val_accuracy: 0.6000 - val_loss: 0.

Test accuracy: 0.7000
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
              precision    recall  f1-score   support

           0       0.71      0.71      0.71        21
           1       0.68      0.68      0.68        19

    accuracy                           0.70        40
   macro avg       0.70      0.70      0.70        40
weighted avg       0.70      0.70      0.70        40

